In [ ]:
import sys
sys.path.append("..") # Adds the module to path

# deeptrack.image

This notebook introduces the module deeptrack.image.

## What is an image?

Images are the containers that are used by features to store both the image its generating and the properties it used to generate the image. It is a subclass of numpy ndarray, meaning any operation that works on numpy ndarrays will also work in instances of Image. However, Image also has a field `properties`, which contains the information about how the image was generated. Specifically, it's a list of dictionaries, where each dictionary is the current values of the properties of a feature (with the names of the properties as keys). The list is ordered in the order the features have been resolved.

By storing the properties used to resolve the image, information about the image can be accessed without access to the feature series. This allows features to change their behaviour depending on what is already in the image. It also makes it possible to extract numeric information about the image used in machine learning. 

In [ ]:
import numpy as np

from deeptrack.features import Feature
from deeptrack.image import Image

class Particle(Feature):
    def get(self, image, position=None, **kwargs):
        
        # Code for simulating a particle not included
        
        return image
    
image_shape = (256,256)
    
particle = Particle(
    position= lambda: np.random.rand(2) * np.array(image_shape)
)



input_image = Image(np.zeros(image_shape))

output_image = particle.resolve(input_image)

output_image.properties

## Storing metadata about an image not used by the feature

Sometimes it's convenient to store some information about a feature outside what is necessary to generate it. For example, you may want to specify one particle to track, or you might want to randomize the diffusion-constant of a particle to correctly generate sequences of images. This can be done by passing more keyword arguments to the constructor of a feature. These will be stored as properties just like any other input to the constructor.

In [ ]:
p_1 = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    track_me=True
)

p_2 = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    track_me=False
)

input_image = Image(np.zeros(image_shape))

particles = p_2**4 + p_1

output_image = particles.resolve(input_image, a=1)

output_image.properties

In [ ]:
particle = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    diffusion=lambda: 1 + np.random.rand() * 4
) 

input_image = Image(np.zeros(image_shape))

particles = particle ** 3

output_image = particles.resolve(input_image)

output_image.properties